## Creating infrastructure for the AgentCore Runtime (IAM, ECR)

In [1]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "strands_claude_getting_started"

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

response

Entrypoint parsed: file=/Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial/strands_claude.py, bedrock_agentcore_name=strands_claude
Configuring BedrockAgentCore agent: strands_claude_getting_started
Generated Dockerfile: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial/Dockerfile
Generated .dockerignore: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial/.dockerignore
Keeping 'strands_claude_getting_started' as default agent
Bedrock AgentCore configured: /Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial/Dockerfile'), dockerignore_path=PosixPath('/Users/ykhvainitski/repos/bedrock-strands/ac_runtime_tutorial/.dockerignore'), runtime='Docker', region='us-east-1', account_id='538213298977', execution_role=None, ecr_repository=None, auto_create_ecr=True)

In [2]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started' to account 538213298977 (us-east-1)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started' to account 538213298977 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_claude_getting_started


✅ ECR repository available: 538213298977.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started
Getting or creating execution role for agent: strands_claude_getting_started
Using AWS region: us-east-1, account ID: 538213298977
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
Starting execution role creation process for agent: strands_claude_getting_started
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
✓ Role created: arn:aws:iam::538213298977:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_claude_getting_started
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::538213298977:role/AmazonBedrockAgentCoreSDKRunt

## Checking the status and make test invocations of the AgentCore Runtime

In [3]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
    
status

Retrieved Bedrock AgentCore status for: strands_claude_getting_started


'READY'

In [4]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})

invoke_response

Invoking BedrockAgentCore agent 'strands_claude_getting_started' via cloud endpoint


{'ResponseMetadata': {'RequestId': 'd13620f9-ebc8-4ab6-a0cd-c1e1de3cd373',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 26 Aug 2025 18:13:20 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd13620f9-ebc8-4ab6-a0cd-c1e1de3cd373',
   'baggage': 'Self=1-68adf93e-1a97b86f4109f8fa5f79176c,session.id=57464fa0-d982-4434-a882-27b9f742fbe3',
   'x-amzn-bedrock-agentcore-runtime-session-id': '57464fa0-d982-4434-a882-27b9f742fbe3',
   'x-amzn-trace-id': 'Root=1-68adf93e-69b888ed016b62370ea3e4b5;Self=1-68adf93e-1a97b86f4109f8fa5f79176c'},
  'RetryAttempts': 0},
 'runtimeSessionId': '57464fa0-d982-4434-a882-27b9f742fbe3',
 'traceId': 'Root=1-68adf93e-69b888ed016b62370ea3e4b5;Self=1-68adf93e-1a97b86f4109f8fa5f79176c',
 'baggage': 'Self=1-68adf93e-1a97b86f4109f8fa5f79176c,session.id=57464fa0-d982-4434-a882-27b9f742fbe3',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'{"result": "

In [10]:
from IPython.display import Markdown, display
import json

response_text = json.loads(invoke_response['response'][0].decode("utf-8"))

display(Markdown(response_text['result']))

The weather is sunny right now.

## Invoking AgentCore Runtime with boto3

In [12]:
import boto3

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]

    response = json.loads(events[0].decode("utf-8"))
    display(Markdown(response['result']))

The expression '2+2' evaluates to 4.

## Cleanup (deleting resources)

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

AttributeError: 'LaunchResult' object has no attribute 'execution_role'

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)